In [3]:
pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [3]:
spark = SparkSession.builder \
    .master("spark://MacBook-Air.local:7077") \
    .appName('test') \
    .getOrCreate()

23/06/15 19:13:17 WARN Utils: Your hostname, MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.68.121 instead (on interface en0)
23/06/15 19:13:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/15 19:13:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [6]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [7]:
df_green = df_green.withColumnRenamed('lpep_pickup_datetime','pickup_datetime').withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')

In [8]:
df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime','pickup_datetime').withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

In [9]:
common_columns=[]
yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [10]:
print(common_columns)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'congestion_surcharge']


In [11]:
from pyspark.sql import functions as F

In [12]:
df_green_sel = df_green \
.select(common_columns) \
.withColumn('service_type',F.lit('green'))

In [13]:
df_yellow_sel = df_yellow \
.select(common_columns) \
.withColumn('service_type',F.lit('yellow'))

In [14]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [15]:
df_trips_data.createOrReplaceTempView('trips_data')

In [16]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [17]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|         112|2020-

In [18]:
df_result.write.parquet('data/report/revenue')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/Shared/Files From c.localized/Data_Engineering_Zoomcamp/Week_5/data/report/revenue already exists. Set mode as "overwrite" to overwrite the existing path.

23/06/15 23:09:05 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.68.121: worker lost
23/06/16 01:55:19 ERROR TaskSchedulerImpl: Lost executor 1 on 192.168.68.121: worker lost
23/06/16 04:56:57 ERROR TaskSchedulerImpl: Lost executor 2 on 192.168.68.121: worker lost
23/06/16 05:32:05 WARN HeartbeatReceiver: Removing executor 3 with no recent heartbeats: 1080141 ms exceeds timeout 120000 ms
23/06/16 05:32:05 ERROR TaskSchedulerImpl: Lost executor 3 on 192.168.68.121: Executor heartbeat timed out after 1080141 ms
23/06/16 07:40:23 ERROR TaskSchedulerImpl: Lost executor 4 on 192.168.68.121: worker lost
23/06/16 12:30:20 WARN HeartbeatReceiver: Removing executor 5 with no recent heartbeats: 1057718 ms exceeds timeout 120000 ms
23/06/16 12:30:20 ERROR TaskSchedulerImpl: Lost executor 5 on 192.168.68.121: Executor heartbeat timed out after 1057718 ms
23/06/16 15:58:18 ERROR TaskSchedulerImpl: Lost executor 6 on 192.168.68.121: worker lost
